In [32]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# Decision Tree Regressor

In [12]:
class NodeRegressor():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value

In [45]:
class DecisionTreeRegressor:
    def __init__(self, min_samples_split=2, max_depth=2, n_feats=None):
        self.root = None
        # stopping conditions for regularization
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats=n_feats
    
    def fit(self,X,y):
        dataset = np.concatenate((X,y),axis=1)
        self.root = self.build_tree(dataset)
    
    def variance_reduction(self,parent,left,right):
        weight_l = len(left)/len(parent)
        weight_r = len(right)/len(parent)
        var_red = np.var(parent)-(weight_l*np.var(left)+weight_r*np.var(right))
        return var_red
    
    def calculate_leaf_value(self, y):
        mean_y = np.mean(y)
        return mean_y
        
    def build_tree(self,dataset,curr_depth=0):
        x = dataset[:,:-1]
        y = dataset[:,-1]
        n_labels = len(np.unique(y))
        num_examples,num_features=x.shape
        if curr_depth<=self.max_depth and num_features>=self.min_samples_split:
            if self.n_feats is None:
                feat_idxs = np.arange(num_features)
            else:
                feat_idxs = np.random.choice(num_features, self.n_feats, replace=False)
            best_split=self.get_best_split(dataset,num_examples,feat_idxs)
            if best_split!={} and best_split["var_red"]>0:
                feature_index=best_split["feature_index"]
                threshold=best_split["threshold"]
                left=best_split["left"] 
                right=best_split["right"]
                var_red=best_split["var_red"]
                left_subtree=self.build_tree(left,curr_depth+1)
                right_subtree=self.build_tree(right,curr_depth+1)
                return NodeRegressor(feature_index,threshold,left_subtree,right_subtree,var_red)
        leaf_val=self.calculate_leaf_value(y)
        return NodeRegressor(value=leaf_val)

    def get_best_split(self,dataset,num_examples,feat_idxs):
        best_split = {}
        y = dataset[:,-1]
        max_var_red= -float("inf")
        for feature_index in feat_idxs:
            feature_vector = dataset[:,feature_index]
            thresholds = np.unique(feature_vector)
            for threshold in thresholds:
                dataset_left,dataset_right = self.split(dataset,feature_index,threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y_left, y_right = dataset_left[:,-1],dataset_right[:,-1]
                    curr_var_red = self.variance_reduction(y,y_left,y_right)
                    if max_var_red<curr_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["left"] =dataset_left
                        best_split["right"]=dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red =curr_var_red
        return best_split
    
    def split(self,dataset,feature_index,threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def print_tree(self,tree=None, indent=" "):
        
        if tree==None:
            tree=self.root
        
        if tree.value is None:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "? info_gain: ", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
        else:
            print(tree.value)
    
    def predict(self,x):
        
        preds = np.array([self.make_prediction(xi,self.root) for xi in x])
        return preds
    
    def make_prediction(self,x,tree):
        if tree.value is not None:
            return tree.value
        threshold = tree.threshold
        if x[tree.feature_index] <= threshold:
            return self.make_prediction(x,tree.left)
        else:
            return self.make_prediction(x,tree.right)


# Decision Tree Classifier

In [14]:
class NodeClassifier():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [36]:
class DecisionTreeClassifier:
    def __init__(self, min_samples_split=2, max_depth=2, n_feats=None,mode="entropy"):
        self.root = None
        # stopping conditions for regularization
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.mode=mode
    
    def fit(self,X,y):
        dataset = np.concatenate((X,y),axis=1)
        self.root = self.build_tree(dataset)
    
    def information_gain(self,parent,left,right):
        weight_l = len(left)/len(parent)
        weight_r = len(right)/len(parent)
        if self.mode=="gini":
            info_gain = self.gini_index(parent)-(weight_l*self.gini_index(left)+weight_r*self.gini_index(right))
        else:
            info_gain = self.entropy(parent)-(weight_l*self.entropy(left)+weight_r*self.entropy(right))
        
        return info_gain
        
    def entropy(self,y):
        entropy=0
        classes = np.unique(y)
        for c in classes:
            prob = len(y[y==c])/len(y)
            entropy += -prob*np.log2(prob)
        return entropy
    
    def gini_index(self,y):
        gini=0
        classes = np.unique(y)
        for c in classes:
            prob = len(y[y==c])/len(y)
            gini += prob**2
        return 1-gini
    
    def calculate_leaf_value(self, y):
        max_y = Counter(y).most_common(1)[0][0]
        return max_y
        
    def build_tree(self,dataset,curr_depth=0):
        x = dataset[:,:-1]
        y = dataset[:,-1]
        n_labels = len(np.unique(y))
        num_examples,num_features=x.shape
        
        if curr_depth<=self.max_depth and n_labels > 1 and num_features>=self.min_samples_split:
            if self.n_feats is None:
                feat_idxs = np.arange(num_features)
            else:
                feat_idxs = np.random.choice(num_features, self.n_feats, replace=False)

            best_split=self.get_best_split(dataset,num_examples,feat_idxs)
            if best_split!={} and best_split["info_gain"]>0:
                feature_index=best_split["feature_index"]
                threshold=best_split["threshold"]
                left=best_split["left"] 
                right=best_split["right"]
                info_gain=best_split["info_gain"]
                left_subtree=self.build_tree(left,curr_depth+1)
                right_subtree=self.build_tree(right,curr_depth+1)
                return NodeClassifier(feature_index,threshold,left_subtree,right_subtree,info_gain)
        leaf_val=self.calculate_leaf_value(y)
        return NodeClassifier(value=leaf_val)

    def get_best_split(self,dataset,num_examples,feat_idxs):
        best_split = {}
        y = dataset[:,-1]
        max_gain= -float("inf")
        for feature_index in feat_idxs:
            feature_vector = dataset[:,feature_index]
            thresholds = np.unique(feature_vector)
            for threshold in thresholds:
                dataset_left,dataset_right = self.split(dataset,feature_index,threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y_left, y_right = dataset_left[:,-1],dataset_right[:,-1]
                    curr_gain = self.information_gain(y,y_left,y_right)
                    if max_gain<curr_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["left"] =dataset_left
                        best_split["right"]=dataset_right
                        best_split["info_gain"] = curr_gain
                        max_gain=curr_gain
        return best_split
    
    def split(self,dataset,feature_index,threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def print_tree(self,tree=None, indent=" "):
        
        if tree==None:
            tree=self.root
        
        if tree.value is None:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "? info_gain: ", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
        else:
            print(tree.value)
    
    def predict(self,x):
        
        preds = np.array([self.make_prediction(xi,self.root) for xi in x])
        return preds
    
    def make_prediction(self,x,tree):
        if tree.value is not None:
            return tree.value
        threshold = tree.threshold
        if x[tree.feature_index] <= threshold:
            return self.make_prediction(x,tree.left)
        else:
            return self.make_prediction(x,tree.right)


# Random Forests

In [26]:
class RandomForests:
    def __init__(self, n_trees=10, min_samples_split=2, max_depth=100, n_feats=None, is_classifier=True):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []
        self.is_classifier = is_classifier
    
    def bootstrap(self,x,y):
        n_samples = x.shape[0]
        idxs = np.random.choice(n_samples,n_samples,replace=True)
        return x[idxs], y[idxs]
        
    def fit(self,x,y):
        for _ in range(self.n_trees):
            if self.is_classifier:
                tree = DecisionTreeClassifier(
                    min_samples_split=self.min_samples_split,
                    max_depth=self.max_depth,
                    n_feats=self.n_feats,
                )
            else:
                tree = DecisionTreeRegressor(
                    min_samples_split=self.min_samples_split,
                    max_depth=self.max_depth,
                    n_feats=self.n_feats
                )
            X_samp, y_samp = self.bootstrap(x, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)
    def predict(self,x):
        all_preds = np.array([tree.predict(x) for tree in self.trees])
        all_preds = np.swapaxes(all_preds,0, 1)
        if self.is_classifier:
            all_preds = np.array([Counter(pred).most_common(1)[0][0] for pred in all_preds])
        else:
            all_preds = np.array([np.mean(pred) for pred in all_preds])
        return all_preds

# Test Random Forests

## Classification

In [27]:
df = pd.read_csv("./Decision_Trees/Breast_cancer_data.csv")

In [28]:
df.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0


In [29]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [33]:
n_feats = np.sqrt(X_train.shape[1])

In [34]:
n_feats, X.shape[1]

(2.23606797749979, 5)

In [37]:
my_classifier = RandomForests(min_samples_split=3, max_depth=3,n_trees=5, n_feats=int(n_feats))
my_classifier.fit(X_train,Y_train)

In [38]:
Y_pred = my_classifier.predict(X_test) 
accuracy_score(Y_test, Y_pred)

0.9298245614035088

### Sklearn

In [42]:
sklearn_rf_clf = RandomForestClassifier(n_estimators=5, criterion='entropy', max_depth=3, min_samples_split=3)
sklearn_rf_clf.fit(X_train,Y_train)
sklearn_clf_preds = sklearn_rf_clf.predict(X_test)
accuracy_score(Y_test, sklearn_clf_preds)

C:\Users\001\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.9385964912280702

## Regression

In [43]:
df = pd.read_csv("./Decision_Trees/airfoil_noise_data.csv")
df.head()

,x0,x1,x2,x3,x4,y
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


In [44]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [46]:
n_feats = np.sqrt(X_train.shape[1])

In [47]:
n_feats,X.shape[1]

(2.23606797749979, 5)

In [48]:
my_regressor = RandomForests(min_samples_split=3, max_depth=3,n_trees=5, n_feats=int(n_feats),is_classifier=False)
my_regressor.fit(X_train,Y_train)

In [50]:
Y_pred = my_regressor.predict(X_test) 
mean_squared_error(Y_test,Y_pred)

20.834140870775546

### Sklearn

In [51]:
sklearn_rf_reg = RandomForestRegressor(n_estimators=5, max_depth=3, min_samples_split=3)
sklearn_rf_reg.fit(X_train,Y_train)
sklearn_reg_preds = sklearn_rf_reg.predict(X_test)
mean_squared_error(Y_test, sklearn_reg_preds)

C:\Users\001\anaconda3\envs\py310\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


24.533940655721107